In [1]:
import os
os.chdir('/Users/layne/Desktop/aiqc')
os.getcwd()

'/Users/layne/Desktop/aiqc'

In [2]:
import aiqc
from aiqc import examples

/Users/layne/.pyenv/versions/3.7.6/envs/jupyterlab/lib/python3.7/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import *

In [4]:
from importlib import reload
aiqc.delete_db(True)
reload(aiqc)
aiqc.create_db()


=> Success - deleted database file at path:
/Users/layne/Library/Application Support/aiqc/aiqc.sqlite3


=> Success - created database file at path:
/Users/layne/Library/Application Support/aiqc/aiqc.sqlite3


=> Success - created the following tables within database:
['algorithm', 'batch', 'dataset', 'encoderset', 'featurecoder', 'featureset', 'file', 'fold', 'foldset', 'hyperparamcombo', 'hyperparamset', 'image', 'job', 'jobset', 'label', 'labelcoder', 'result', 'splitset', 'tabular']



---

In [200]:
dataset = aiqc.Dataset.Tabular.from_path(
    examples.get_demo_file_path('iris.tsv')
    , source_file_format = 'tsv'
)

In [201]:
dataset.to_pandas().head().dtypes

sepal_length    float64
sepal_width     float64
petal_length    float64
petal_width     float64
species           int64
dtype: object

In [202]:
label = dataset.make_label(columns=['kingdom_category'])

ValueError: 
Yikes - You specified `columns` that do not exist in the Dataset.


In [183]:
featureset = dataset.make_featureset(exclude_columns=[
    'kingdom_category', 'radiance_float32', 'temperature_float', 'doom_float64'
])

In [184]:
splitset = featureset.make_splitset(
    label_id=label.id
    , size_test = 0.22
    , size_validation = 0.12 
)

---

In [185]:
encoderset = splitset.make_encoderset()

In [186]:
labelcoder = encoderset.make_labelcoder(
    sklearn_preprocess = OneHotEncoder(sparse=False)
    , only_fit_train = False
)

---
---
---

In [187]:
# fc = encoderset.make_featurecoder(
# 	sklearn_preprocess = StandardScaler()
# 	, only_fit_train = True
# 	, include = True
# 	, dtypes = ['float32']
# 	, columns = None
# )

In [188]:
# fc = encoderset.make_featurecoder(
# 	sklearn_preprocess = OrdinalEncoder()
# 	, only_fit_train = False
# 	, include = True
# 	, dtypes = None
# 	, columns = ['hour_object', 'month_int32', 'moonday_int64', "weekday_object"]
# )

In [189]:
# fc = encoderset.make_featurecoder(
# 	sklearn_preprocess = OneHotEncoder(sparse=False)
# 	, only_fit_train = False
# 	, include = True
# 	, dtypes = ['uint8', 'int64']
# 	, columns = None
# )

---
---
---

In [190]:
import keras
from keras import metrics
from keras.models import Sequential
from keras.callbacks import History
from keras.layers import Dense, Dropout

In [191]:
len(featureset.columns)

6

In [192]:
def multiclass_function_model_build(**hyperparameters):
	model = Sequential()
	model.add(Dense(hyperparameters['neuron_count'], input_shape=(9,), activation='relu', kernel_initializer='he_uniform'))
	model.add(Dropout(0.2))
	model.add(Dense(hyperparameters['neuron_count'], activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(3, activation='softmax'))

	opt = keras.optimizers.Adamax(hyperparameters['learning_rate'])
	model.compile(
		loss = 'categorical_crossentropy'
		, optimizer = opt
		, metrics = ['accuracy']
	)
	return model

In [193]:
def multiclass_function_model_train(model, samples_train, samples_evaluate, **hyperparameters):
	model.fit(
		samples_train["features"]
		, samples_train["labels"]
		, validation_data = (
			samples_evaluate["features"]
			, samples_evaluate["labels"]
		)
		, verbose = 0
		, batch_size = hyperparameters['batch_size']
		, epochs = hyperparameters['epoch_count']
		, callbacks=[History()]
	)
	return model

In [194]:
algorithm = aiqc.Algorithm.make(
    library = "keras"
    , analysis_type = "classification_multi"
    , function_model_build = multiclass_function_model_build
    , function_model_train = multiclass_function_model_train
)

In [195]:
hyperparameters = {
    "neuron_count": [9, 12]
    , "epoch_count": [30, 60]
    , "learning_rate": [0.01]
    , "batch_size": [3, 5]
}

hyperparamset = aiqc.Hyperparamset.from_algorithm(
    algorithm_id = algorithm.id
    , hyperparameters = hyperparameters
)

In [196]:
batch = aiqc.Batch.from_algorithm(
    algorithm_id = algorithm.id
    , splitset_id = splitset.id
    , hyperparamset_id = hyperparamset.id
    , foldset_id = None
    , encoderset_id = encoderset.id
)